In [5]:
import os
import sys
import shutil
import glob
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from itertools import permutations
from multiprocessing import Pool

In [4]:
data_dir = "/data5/galaxy/project/expression/DESeq2"
positive_pairs = glob.glob("%s/*.csv" % data_dir)
result_list, result_file = [], "/data5/galaxy/project/expression/featureCounts/input/paired_tissue.txt"
for pair in positive_pairs:
    tissue_1 = os.path.basename(pair).split("_")[0]
    tissue_2 = os.path.basename(pair).split("_")[-1].split(".")[0]
    result_list.append("%s\t%s\n" % (tissue_2, tissue_1))
with open(result_file, 'w') as fw:
    fw.write("tissue1\ttissue2\n")
    fw.writelines(result_list)

16

In [40]:
data_dir = "/data5/galaxy/project/expre_vs_m6a/expre_vs_m6a_2"
os.chdir(data_dir)
file_list, file_dict = glob.glob("*.txt"), {}
for x in file_list:
    tissue1, tissue2 = x.split("_")[0], x.split("_")[2].split("-")[0]
    file_dict[tissue1] = file_dict.get(tissue1, []) + [tissue2]
for a, b in file_dict.items():
    file_dict[a] = set(b)
#
ordered_list = get_ordered_list(file_dict)
###
for gene_type in ["gene_up", "gene_down"]:
    result_file = "format_txt/data-%s.txt" % gene_type
    if os.path.exists(result_file):
        os.remove(result_file)
    with open(result_file, 'a+') as fw:
        for tissue1 in ordered_list:
            tissue2_list = file_dict[tissue1]
            title_list, up_list, down_list, common_list = [tissue1], [""], [""], [""]
            for tissue_2 in tissue2_list:
                title_list.append(tissue_2)
                up_num, down_num, common_num = process_data(tissue1, tissue_2, gene_type)
                up_list.append(up_num)
                down_list.append(down_num)
                common_list.append(common_num)
            for i_list in [title_list, up_list, down_list, common_list]:
                fw.write("%s\n" % "\t".join([str(x) for x in i_list]))

39

33

20

28

35

...

8

11

4

6

5

In [37]:
def get_ordered_list(file_dict):
    key_list, value_list = [], []
    for x, i_list in file_dict.items():
        key_list.append(x)
        value_list.append(len(i_list))
    index_list = list(np.argsort(value_list))
    index_list.reverse()
    ordered_list = [key_list[i] for i in index_list]
    return ordered_list

def process_data(t_1, t_2, gene_type):
    prefix = "%s_vs_%s-%s" % (t_1, t_2, gene_type)
    m6a_up = "%s-m6a_up.txt" % prefix
    m6a_down = "%s-m6a_down.txt" % prefix
    m6a_common = "%s-m6a_common.txt" % prefix
    up_num, down_num, common_num = stat_line_num(m6a_up), stat_line_num(m6a_down), stat_line_num(m6a_common)
    return up_num, down_num, common_num

    
def stat_line_num(in_file):
    with open(in_file, 'r') as f:
        return len(f.readlines())

In [24]:
a = (1,2,3,3)
b = (5)
a

TypeError: can only concatenate tuple (not "int") to tuple